# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to Load
cities = "cities_cleaned.csv"

# Read Purchasing File and store into Pandas data frame
cities_data = pd.read_csv(cities)
cities_data.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,São João da Barra,-21.6403,-41.0511,295.14,94,72,4.93,BR,1637212685
1,Kenai,60.5544,-151.2583,261.56,77,1,0.00,US,1637212685
2,Enugu,6.4402,7.4943,295.93,94,5,0.97,NG,1637212686
3,Kavieng,-2.5744,150.7967,302.19,69,9,4.13,PG,1637212686
4,Port Hedland,-20.3167,118.5667,306.18,49,14,9.26,AU,1637212686


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
cities_data = cities_data.dropna()
humidity = cities_data["Humidity"].astype(float)

# Store 'Lat' and 'Long' into  locations. 
locations = cities_data[["Lat", "Long"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(cities_data["Humidity"]),
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
# Narrow down the DataFrame to find your ideal weather condition.
ideal_weather = cities_data

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
ideal_weather = ideal_weather.loc[(ideal_weather["Max Temp"] < 305) & (ideal_weather["Max Temp"] > 298)]

# Wind speed less than 10 mph.
ideal_weather = ideal_weather.loc[ideal_weather["Wind Speed"] < 10]

# Zero cloudiness.
ideal_weather = ideal_weather.loc[ideal_weather["Cloudiness"] == 0]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna()

ideal_weather

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
13,Seddon,-37.8068,144.8916,301.59,36,0,7.15,AU,1637212689
38,Saint-Pierre,-21.3393,55.4781,301.97,55,0,3.60,RE,1637212698
113,San Patricio,28.0170,-97.5169,298.07,70,0,5.22,US,1637212723
206,Dubai,25.2582,55.3047,300.29,61,0,4.63,AE,1637212755
317,Salalah,17.0151,54.0924,300.20,36,0,1.03,OM,1637212793
435,Beloha,-25.1667,45.0500,303.77,30,0,1.69,MG,1637212835


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = ideal_weather

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Long"]

    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field or result... skipping.")
        
    print("------------")

Retrieving Results for Index 13: Seddon.
Closest hotel is Footscray Motor Inn.
------------
Retrieving Results for Index 38: Saint-Pierre.
Closest hotel is Lindsey Hôtel.
------------
Retrieving Results for Index 113: San Patricio.
Closest hotel is Motel 6 Sinton, TX.
------------
Retrieving Results for Index 206: Dubai.
Closest hotel is Radisson Blu Hotel, Dubai Deira Creek.
------------
Retrieving Results for Index 317: Salalah.
Closest hotel is HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel.
------------
Retrieving Results for Index 435: Beloha.
Missing field or result... skipping.
------------


In [16]:
hotel_df

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
13,Seddon,-37.8068,144.8916,301.59,36,0,7.15,AU,1637212689,Footscray Motor Inn
38,Saint-Pierre,-21.3393,55.4781,301.97,55,0,3.60,RE,1637212698,Lindsey Hôtel
113,San Patricio,28.0170,-97.5169,298.07,70,0,5.22,US,1637212723,"Motel 6 Sinton, TX"
206,Dubai,25.2582,55.3047,300.29,61,0,4.63,AE,1637212755,"Radisson Blu Hotel, Dubai Deira Creek"
317,Salalah,17.0151,54.0924,300.20,36,0,1.03,OM,1637212793,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
435,Beloha,-25.1667,45.0500,303.77,30,0,1.69,MG,1637212835,NaN


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [21]:
# Add marker layer ontop of heat map
# Display figure
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))